# Модуль Б Разработка модели машинного обучения

## Разработка алгоритма обучения

Моя задача в рамках этого модуя разработать и обучить нейронную сеть на основе подготовленных данных в прошлом модуле которая определит и распознает автомобильные номера российского образца.

Эту задачу можно разделить на несколько частей как уже было сказано: определение и распознавание. Определение как таковое является задачей детекции предмета на изображении.

Пример детекции:

![](./static/val_batch0_labels.jpg)

Модель не только находит и классифицирует предмет на картинке, но и обводит его в рамочку, которую можно использовать в дальнейших целях.

Таким образом в нашей задаче я буду использовать модель детекции yolov5, работу которой вы можете увидеть на картинке выше

После того как моя модель определит наш номер и поставит для него рамочку мы обрежем картинку по этой рамочке и будем работать с этим изображением для распознавания номера автомобиля, ведь без такой обрезки модель чтения текста с картинки может либо прочитать что-то на изображении что не относится к автомобильному номеру, либо в общем, из-за нагруженности изображения не увидеть номер и/или прочитать не правильно.

Говоря о распознавании автомобильного номера на картинке есть несколько способов прочитать текст с изображения. Один из них я буду использовать - это предобученная библиотека easyocr. Своего рода готовое решение в библиотеке я беру не с проста. Языковые модели которые используются при чтения данных с изображений при обучении требуют огромное количество ресурсов, по сравнению с уже обученной моделью. Для таких тяжёлых нейросетей нужно большое количество качественной быстрой оперативной памяти либо такое же большое количество видео памяти с дискретной видеокартой в разы ускоряющей обучение модели. Такими ресурсами моноблок, который я использую не обладает, следовательно остаётся использовать заготовленные библиотеки чтения текста с картинок в угоду времени и во избежание непосильных нагрузок на компьютер

### Подбор параметров

В ходе попыток обучения модели yolov5 и использовании различных библиотек по чтению текста с картинок я подобрал для обеих решений лучшие параметры.

#### Yolov5

Обучение модели yolov5 выглядит примерно следующим образом как можно увидеть в файле ./yolov5/tutorial.ipynb поставляющимся вместе с самой моделью:

![](./static/2.png)

после команд "!python train.py" идут всеразличные параметры по изменению нашей модели:
- размер картинок на вход
- сколько картинок за раз поставляется на вход
- сколько раз нейронная сеть должна пройти по всем данным
и так далее

даже в этом примере параметров много, но изучив код обучения модели я пошёл дальше и в ходе тестирования моделей(запуска различных параметров в подмодуле 3 обучения модели), анализа их метрик ошибок(которые вы можете увидеть и в картинке выше с приставками '_loss') и в попытке их уменьшения я пришёл к следующим параметрам:

In [1]:
!python train.py --img 244 --batch 10 --epochs 10 --data dataset.yaml --weights yolov5n.pt --cache --nosave --optimizer Adam --workers 0 --label-smoothing 0.0 --patience 5 --freeze 0 

python: can't open file 'C:\\Users\\Participant\\Desktop\\Rudnev\\Module b\\train.py': [Errno 2] No such file or directory


--img Размер изображений на вход я уменьшил, т.к. большие фотографии занимают много памяти и кроме того загрузка фотографий базового размера в 640 на компьютере требует на много больше времени
--batch Количество на вход картинок я уменьшил вдвое, для того чтобы уменьшить затраты памяти компьютера, чтобы модель в последствии и её обучение не требовали много памяти. см. след. картинку 

![](./static/3.png)

4642,3 МБ приемлемо для обучения и даже дальнейшей работы с кодом пока модель обучается, кроме того большую часть из этих 50% использует pycharm в обычном режиме

--epochs 10 я поставил в угоду точности модели, ведь именно с таким параметром она выдавала приемлемый результат за потраченное время
--data параметр загрузки своих данных, в который я указал dataset.yaml - файл полной загрузки наших данных созданный в последующих модулях

Остальными параметрами я оперировал в зависимости от ошибок модели и её точности.

### Модель распознавания

Готовых моделей распознавания не так много, по крайней мере не так много на самом деле точных моделей для нашей задачи распознавания номеров.

Я пробовал разные модели на данных которые сохранил себе в папке .\Result_A\cropped. и наиболее валидной для нашей задачи оказалась модель библиотеки easyocr

Работа модели:

In [2]:
import easyocr # Импортирование класса easyocr содержащий модель
reader = easyocr.Reader(['en','ru'],gpu = False) # обозначение модели читающей текст с картинки
result = reader.readtext('./Result_A/cropped/7.jpg') # работа модели по прочтению текста
result[0][1]


KeyboardInterrupt



In [ ]:
result[1][1]

Как мы видим например на случайной картинке в нашем датасете ./Result_A/cropped/ модель всё распознала верно

![](./Result_A/cropped/7.jpg)

для такой модели подбор параметров значительно мал по сравнению с той же yolov5, но в угоду точности я использовал два языка прочтения ['en','ru'](английский и русский в easyocr.Reader(['en','ru'],gpu = False)) и использовал параметр gpu=False, т.к. этот параметр ставится True когда в устройстве есть видеокарта для ускорения

надпись: Using CPU. Note: This module is much faster with a GPU. это как раз оповещение, что модель использует именно центральный процессор а не видеокарту которой нет.

## Импорт заготовленных данных

Из прошлого модуля А мы получили в результате папку Result_A, которая содержит данные, нужные для обучения наших моделей. В этом подмодуле мы должны импортировать их, преобразовать и внести в тренировку нейросетевых моделей, которые я буду использовать

Для начала напишем код, который разделит наши данные в папке Result_A\licenses тоесть labels и images на 2 выборки, обучающую и валидационную, потому что именно такой формат данных принимает модель yolov5.

In [ ]:
from glob import glob
import random
files_img = glob('./Result_A/licenses/images/*.jpg')
random.shuffle(files_img)
files_txt = glob('./Result_A/licenses/labels/*.txt')
random.shuffle(files_txt)
print('Файлы с изображениями: ',len(files_img), 'Файлы с разметками: ',len(files_txt))

Как видим мы импортируем __все__ файлы с разметками и изображениями из этой директории ведь даже файловая система windows показывает что их 10536:

![](./static/1.png)

Теперь когда мы полностью импортировали данные мы разделим их как я уже сказал на выборки train(80%) val(20%):

In [ ]:
training_data = list(map(lambda x:os.path.basename(x)[:-4],files_img[:len(files_img)//100*80]))
validation_data = list(map(lambda x:os.path.basename(x)[:-4],files_img[len(files_img)//100*80:]))

In [ ]:
# И создав папки
import os
path = './data/'
for dir1 in ['train','val']:
    for dir2 in ['labels','images']:
        os.makedirs(os.path.join(path,dir1,dir2),exist_ok=True)

In [ ]:
# Запишем эти выборки
import shutil
src_path = './Result_A/licenses/'
dst_path = './data/'
for train in training_data: # Запись тестовой выборки
    shutil.copy(os.path.join(src_path,'images',train) + '.jpg',os.path.join(path,'train','images'))
    shutil.copy(os.path.join(src_path,'labels',train) + '.txt',os.path.join(path,'train','labels'))
for val in validation_data: # Запись валидационной выборки
    shutil.copy(os.path.join(src_path,'images',val) + '.jpg',os.path.join(path,'val','images'))
    shutil.copy(os.path.join(src_path,'labels',val) + '.txt',os.path.join(path,'val','labels'))

Так же, для использования, полностью импортируем и датасеты из ./Result_A/cropped и ./Result_A/symbols

In [ ]:
cropped_data = glob('./Result_A/cropped/*')
print('Данные из ./Result_A/cropped: ', len(cropped_data))

In [ ]:
path = './Result_A/symbols/'
symbols_data = []
for class_num in range(0,22):
    symbols_data.append(glob(path+str(class_num)+'/*'))
summa = 0
for class_num in range(0,22):
    summa += len(symbols_data[class_num])
print('Данные из ./Result_A/symbols: ', summa)

Как можем увидеть файлов такое же количество в cropped и в symbols:

![](./static/4.png)

Т.к. они импортированы выведем их в виде изображений:

In [ ]:
import matplotlib.pyplot as plt
import random
print('Данные cropped:')
plt.imshow(plt.imread(cropped_data[random.randint(0,len(cropped_data))]))

In [ ]:
print('Данные symbols:')
plt.imshow(plt.imread(symbols_data[0][random.randint(0,len(symbols_data[0]))]))

In [ ]:
print('Данные licenses:')
plt.imshow(plt.imread(files_img[random.randint(0,len(files_img))]))

Никаких ошибок импорта замечено не было.

# Обучение нейросети

Для начала чтобы обучить нашу нейросеть нужно создать файл импорта наших данных в модель при её обучении

Примерами таких файлов являются данные с расширением .yaml в директории ./yolov5/data, по этому примеру создаём наш файл для импорта всех данных:

In [ ]:
import yaml
data = {
    'train': '../data/train',
    'val': '../data/val',
    'names': {0 : 'license_plate'}
}
with open('./yolov5/data/dataset.yaml','w') as f:
    yaml.dump(data,f)

далее можем запустить модель с помощью команды: python train.py --epochs 1 --img 640 --batch 8 --data dataset.yaml --weights yolov5n.pt --cache и проверить что наши данные из разделённого на подкаталоги датасета из заданных данных загружаются в модель без ошибок и полностью:

![](./static/5.png)

### Код обучения нейросети:

Модель yolov5 обучается на предобработанных данных с определённой структурой через команду в терминале, которая вызывает train.py и назначает для обучения различные параметры.

для того чтобы вызвать train.py и назначить остальные параметры перейдём в директорию ./yolov5 с помощью команды в терминале:

cd yolov5

в этом же терминале __*согласно нашему алгоритму обучения*__ введём команду которая начнёт обучение модели:

In [ ]:
!python train.py '''файл для обучения''' --img 244 '''размер изображений''' --batch 10 '''сколько изображений за один раз(батч)''' --epochs 10 '''сколько эпох модель будет обучаться''' --data dataset.yaml '''файл импорта данных''' --weights yolov5n.pt '''веса модели, которые обучаются''' --cache '''сохранение кэша данных для лучшей работы модели''' --nosave '''сохранение только последних лучших весов модели(чтобы не использовать много памяти)''' --optimizer Adam '''назначение оптимизатора модели''' --workers 0 '''сколько устройст будет применятся в загрузке(0 - все которые способны на это)''' --label-smoothing 0.0 '''параметр сглаживания класса в модели''' --patience 5 '''параметр 'терпения' модели(если на n количество эпох ошибка не уменьшилась то модель сохраняет веса)''' --freeze 0 '''заморозка некоторых слоёв модели(0 - не замораживать слои)'''

в терминале это должно выглядеть вот так:

![](./static/6.png)

И для выхода из директории yolov5 после обучения используем команду:

cd ../

Как я уже говорил при разработке алгоритма обучения я старался снизить количество используемой памяти моделью разными способами. Поэтому загруженная __модель не использует слишком большое количество памяти__. Кроме того, __при обучении никаких ошибок и предупреждений не возникало__.

Использование памяти из 16 возможных ГБ:

![](./static/8.png)

И использование оперативной памяти когда модель импортирована:

Импорт модели:

In [ ]:
import torch
model = torch.hub.load('./yolov5', 'custom', path='./yolov5/runs/train/exp6/weights/last.pt' ,source='local')  
im = './Result_A/licenses/images/1_11_2014_12_26_41_293.jpg'  
results = model(im)  
results.show() # or .show(), .save(), .crop(), .pandas(), etc.

Как видим оперативной памяти модель использует очень малое количество, вот в сравнении pycharm с заруженной моделью и без:

![](./static/9.png)

порядка 1.2 ГБ

Обучение:

![](./static/7.png)

Обучение прошло без ошибок, результат сохранился в директорию ./yolov5/runs/train/exp6/weights/last.pt(директория exp6 потому что в предыдущих exp-exp5 проводились тесты модели для подбора параметров)

Как можем убедиться на картинке модель обучилась с хорошей точностью для узких и не самых лучших параметров(в угоду скорости и во избежание затрат памяти)

![](./yolov5/runs/train/exp6/train_batch0.jpg)

Сохраним файл обученной модели

In [ ]:
import pickle
model = torch.hub.load('./yolov5', 'custom', path='./yolov5/runs/train/exp6/weights/last.pt' ,source='local')  
with open('./model.pickle','wb') as f:
    pickle.dump(model,f)

И скопируем веса нашей модели в основную папку модуля

In [ ]:
import shutil
shutil.copy('./yolov5/runs/train/exp6/weights/last.pt','./')

# Разработка и создание API

### Получение изображения с помощью функции

Получение изображения я сделал по пути к самому изображению. В функцию вводится путь к изображению и функция получает данные фото, которые далее выводит в формате массива

In [ ]:
def take_picture(path):
    import cv2
    img = cv2.imread(path)
    return img
result = take_picture(r'./Result_A/licenses/images/1_11_2014_12_30_51_262.jpg')
result

Которую можно потом отобразить и сделать с ней всё что угодно

In [ ]:
plt.imshow(result)

картинка опять чёрно-белая потому что я сдался искать эту настройку с тем что графики не меняют цвет при изменении темы

### Получение результата предсказаний модели

Функция состоит из загрузки в себе двух моделей, кропа изображения полученного из прошлой функции и распознавания текста с помощью easyocr 

In [ ]:
def predicts(img):
    import torch
    import easyocr
    # Кропп изображения
    model = torch.hub.load('./yolov5', 'custom', path='./yolov5/runs/train/exp6/weights/last.pt' ,source='local')# Обозначение модели кропа детекции изображений
    results = model(img) # Детекция номеров на изображении
    if not results.xyxy[0].nelement() == 0:
        crop_xy = list(map(int,(results.xyxy[0].detach().flatten()[0],results.xyxy[0].detach().flatten()[1],results.xyxy[0].detach().flatten()[2],results.xyxy[0].detach().flatten()[3]))) # преобразование результатов детекции в формат xyxy
        img = img[crop_xy[1]:crop_xy[3],crop_xy[0]:crop_xy[2]] # Кропп изображения слайсингом
        # Чтение надписей на изображении
        reader = easyocr.Reader(['en','ru'],gpu = False) # обозначение модели читающей текст с картинки
        result = reader.readtext(img) # работа модели по прочтению текста
        # Преобразование результата в понятную форму и вывод результата
        result = [result[i][1] for i in range(len(result))]
        for res in result:
            print(res)
        return result
    return 'Номер не распознан'
    
result = predicts(take_picture(r'C:\Users\Participant\Desktop\Rudnev\Module b\Result_A\licenses\images\1_11_2014_12_40_55_90.jpg'))
print(result)

Как можем увидеть модель правильно прочитала серию и номер автомобиля

![](.\Result_A\licenses\images\1_11_2014_12_40_55_90.jpg)

### Юнит тестирование

Все юнит тесты производились в директории ./unit в файлах с расширением .py

### Тестирование получения изображения(take_picture_tests.py):

![](./static/10.png)

Вводим команду:

![](./static/11.png)

И получаем ответ от нашего кода что все тесты прошли нормально

![](./static/12.png)

### Также с функцией для вывода результата(prediction_tests.py):

![](./static/13.png)

![](./static/14.png)

![](./static/15.png)

### Приложение

Приложение для распознавания автомобильных номеров было создано в директории ./api на библиотеке streamlit

Для запуска понадобятся нужные для распознавания библиотеки, всё содержимое папки api и команда следующего характера:

![](./static/17.png)

Вот так примерно должна выглядеть работающее api:

In [ ]:
![](./static/18.png)

Вводите путь к изображению на компьютере, нажимаете enter и без должной молитвы тут не обойтись

В файле requirements.txt находятся все используемые мной библиотеки, возможно, если вы всё таки захотите открыть мою апи, оно вам понадобится.

In [ ]:
Чтобы установить библиотеки можно прописать: pip install -r requirements.txt помоему 🤔

Либо на моём компьютере откройте и надеюсь сработает

### Документация

Документация к моему апи я создал в директории Rudnev/Module b/

![](./static/сердешки.png)